##PACKAGE IMPORT SECTION

In [3]:
from google.colab import files
uploaded = files.upload()

Saving multi_task_forest1.py to multi_task_forest1 (2).py


In [ ]:
!pip install git+https://github.com/neurodata/treeple.git

In [ ]:
!pip install proglearn

In [20]:
from multi_task_forest1 import MultiTaskForestClassifier
from proglearn.sims import generate_gaussian_parity
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.datasets import cifar10
from skimage.transform import rotate
from proglearn.sims import generate_spirals

#Testing


##XOR/RXOR

In [7]:
X_xor, y_xor = generate_gaussian_parity(1000)
X_rxor, y_rxor = generate_gaussian_parity(1000, angle_params=np.pi / 4)
X_train_xor, X_test_xor, y_train_xor, y_test_xor = train_test_split(X_xor, y_xor, test_size=0.2, random_state=42)
X_train_rxor, X_test_rxor, y_train_rxor, y_test_rxor = train_test_split(X_rxor, y_rxor, test_size=0.2, random_state=42)


###Train on same task and test on the same task (like a base line)

In [8]:
clf = MultiTaskForestClassifier(clf_type="SPORF")
clf.add_task(0, X_train_xor, y_train_xor)
clf.add_task(1, X_train_rxor, y_train_rxor)
clf.fit([0, 1])
acc_xor = clf.score(X_test_xor, y_test_xor, task_id=0)
acc_rxor = clf.score(X_test_rxor, y_test_rxor, task_id=1)

print(f"XOR Accuracy: {acc_xor:.3f}")
print(f"RXOR Accuracy: {acc_rxor:.3f}")

XOR Accuracy: 0.965
RXOR Accuracy: 0.950


###Forward training, backward training, reverse training

In [9]:
results = clf.evaluate_transfer_general(
    forward_train_ids=[0],
    forward_test_id=1,
    backward_train_ids=[0, 1],
    backward_test_ids=[0, 1],
    do_reverse=True
)

print("Transfer Learning Results:")
fwd = results["forward_transfer"]
print(f"\n Forward Transfer:\n  Train on: {fwd['train_on']}\n  Test on: {fwd['test_on']}\n  Accuracy: {fwd['accuracy']:.3f}")

print("\n Backward Transfer:")
for task, stats in results["backward_transfer"].items():
    print(f"  {task}:")
    print(f"    Train on: {stats['train_on']}")
    print(f"    Test on : {stats['test_on']}")
    print(f"    Accuracy: {stats['accuracy']:.3f}")

if "reverse_transfer" in results:
    print("\n Reverse Transfer:")
    for task, stats in results["reverse_transfer"].items():
        print(f"  {task}:")
        print(f"    Train on: {stats['train_on']}")
        print(f"    Test on : {stats['test_on']}")
        print(f"    Accuracy: {stats['accuracy']:.3f}")

Transfer Learning Results:

 Forward Transfer:
  Train on: [0]
  Test on: 1
  Accuracy: 0.535

 Backward Transfer:
  task0:
    Train on: [0, 1]
    Test on : 0
    Accuracy: 0.994
  task1:
    Train on: [0, 1]
    Test on : 1
    Accuracy: 0.991

 Reverse Transfer:
  task0:
    Train on: [1]
    Test on : 0
    Accuracy: 0.460


## CIFAR 10 (Rotation)


In [10]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X = np.concatenate([X_train, X_test])
y = np.concatenate([y_train, y_test]).flatten()

In [11]:
class0_idx = y == 0
X_class0 = X[class0_idx]
y_class0 = y[class0_idx]

X0 = X_class0[:1000]
y0 = np.zeros(len(X0))

X1 = np.array([rotate(img, angle=90, resize=False) for img in X_class0[1000:2000]])
y1 = np.ones(len(X1))

X0 = X0.reshape(X0.shape[0], -1) / 255.0
X1 = X1.reshape(X1.shape[0], -1) / 255.0
X0_train, X0_test, y0_train, y0_test = train_test_split(X0, y0, test_size=0.2, random_state=42)
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.2, random_state=42)

###Train on same task and test on the same task (like a base line)


In [13]:
clf = MultiTaskForestClassifier(clf_type="SPORF")
clf.add_task(0, X0_train, y0_train)
clf.add_task(1, X1_train, y1_train)

clf.fit([0, 1])

acc_task0 = clf.score(X0_test, y0_test, task_id=0)
acc_task1 = clf.score(X1_test, y1_test, task_id=1)

print(f"Task 0 (Original Class 0) Accuracy: {acc_task0:.3f}")
print(f"Task 1 (Rotated Class 0) Accuracy: {acc_task1:.3f}")

Task 0 (Original Class 0) Accuracy: 1.000
Task 1 (Rotated Class 0) Accuracy: 1.000


###Forward training, backward training, reverse training

In [14]:
results = clf.evaluate_transfer_general(
    forward_train_ids=[0],
    forward_test_id=1,
    backward_train_ids=[0, 1],
    backward_test_ids=[0, 1],
    do_reverse=True
)
print("Transfer Learning Results:")
fwd = results["forward_transfer"]
print(f"\n Forward Transfer:\n  Train on: {fwd['train_on']}\n  Test on: {fwd['test_on']}\n  Accuracy: {fwd['accuracy']:.3f}")

print("\n Backward Transfer:")
for task, stats in results["backward_transfer"].items():
    print(f"  {task}:")
    print(f"    Train on: {stats['train_on']}")
    print(f"    Test on : {stats['test_on']}")
    print(f"    Accuracy: {stats['accuracy']:.3f}")

if "reverse_transfer" in results:
    print("\n Reverse Transfer:")
    for task, stats in results["reverse_transfer"].items():
        print(f"  {task}:")
        print(f"    Train on: {stats['train_on']}")
        print(f"    Test on : {stats['test_on']}")
        print(f"    Accuracy: {stats['accuracy']:.3f}")

Transfer Learning Results:

 Forward Transfer:
  Train on: [0]
  Test on: 1
  Accuracy: 0.000

 Backward Transfer:
  task0:
    Train on: [0, 1]
    Test on : 0
    Accuracy: 1.000
  task1:
    Train on: [0, 1]
    Test on : 1
    Accuracy: 1.000

 Reverse Transfer:
  task0:
    Train on: [1]
    Test on : 0
    Accuracy: 0.000


##CIFAR 10 (Randomly pick 3 classes as 3 tasks)

In [17]:
selected_classes = [0, 3, 7]

task_data = {}
for task_id, cls in enumerate(selected_classes):
    idx = y == cls
    X_cls = X[idx][:1000]  # Use 1000 per class for speed
    y_cls = np.full(len(X_cls), task_id)  # Relabel for task (0, 1, 2)

    X_cls = X_cls.reshape(X_cls.shape[0], -1) / 255.0
    X_train_task, X_test_task, y_train_task, y_test_task = train_test_split(X_cls, y_cls, test_size=0.2, random_state=42)

    task_data[task_id] = (X_train_task, X_test_task, y_train_task, y_test_task)

###Train on same task and test on the same task (like a base line)

In [18]:
clf = MultiTaskForestClassifier(clf_type="SPORF")

for task_id in task_data:
    X_train, X_test, y_train, y_test = task_data[task_id]
    clf.add_task(task_id, X_train, y_train)

clf.fit([0, 1, 2])

for task_id in task_data:
    X_test = task_data[task_id][1]
    y_test = task_data[task_id][3]
    acc = clf.score(X_test, y_test, task_id=task_id)
    print(f"Task {task_id} (Class {task_id}) Baseline Accuracy: {acc:.3f}")

Task 0 (Class 0) Baseline Accuracy: 1.000
Task 1 (Class 1) Baseline Accuracy: 1.000
Task 2 (Class 2) Baseline Accuracy: 1.000


###Forward training, backward training, reverse training

In [19]:
results = clf.evaluate_transfer_general(
    forward_train_ids=[0, 1, 2],    # Train on all 3
    forward_test_id=0,
    backward_train_ids=[0, 1, 2],
    backward_test_ids=[0, 1, 2],    # But test on each task individually
    do_reverse=True
)
print("Transfer Learning Results:")
fwd = results["forward_transfer"]
print(f"\n Forward Transfer:\n  Train on: {fwd['train_on']}\n  Test on: {fwd['test_on']}\n  Accuracy: {fwd['accuracy']:.3f}")

print("\n Backward Transfer:")
for task, stats in results["backward_transfer"].items():
    print(f"  {task}:")
    print(f"    Train on: {stats['train_on']}")
    print(f"    Test on : {stats['test_on']}")
    print(f"    Accuracy: {stats['accuracy']:.3f}")

if "reverse_transfer" in results:
    print("\n Reverse Transfer:")
    for task, stats in results["reverse_transfer"].items():
        print(f"  {task}:")
        print(f"    Train on: {stats['train_on']}")
        print(f"    Test on : {stats['test_on']}")
        print(f"    Accuracy: {stats['accuracy']:.3f}")

Transfer Learning Results:

 Forward Transfer:
  Train on: [0, 1, 2]
  Test on: 0
  Accuracy: 1.000

 Backward Transfer:
  task0:
    Train on: [0, 1, 2]
    Test on : 0
    Accuracy: 1.000
  task1:
    Train on: [0, 1, 2]
    Test on : 1
    Accuracy: 1.000
  task2:
    Train on: [0, 1, 2]
    Test on : 2
    Accuracy: 1.000

 Reverse Transfer:
  task0:
    Train on: [0]
    Test on : 0
    Accuracy: 1.000
  task1:
    Train on: [0]
    Test on : 1
    Accuracy: 0.000
  task2:
    Train on: [0]
    Test on : 2
    Accuracy: 0.000


##Sprial 3 & Spiral 5

In [26]:
n_samples = 750
X3, y3 = generate_spirals(n_samples, 3, noise=0.8)
X5, y5 = generate_spirals(n_samples, 5, noise=0.4)
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, test_size=0.2, random_state=42)
X5_train, X5_test, y5_train, y5_test = train_test_split(X5, y5, test_size=0.2, random_state=42)

task_data = {
    0: (X3_train, X3_test, y3_train, y3_test),  # Spiral 3
    1: (X5_train, X5_test, y5_train, y5_test)   # Spiral 5
}

###Train on same task and test on the same task (like a base line)

In [28]:
clf = MultiTaskForestClassifier(clf_type="SPORF")

for task_id in task_data:
    X_train, X_test, y_train, y_test = task_data[task_id]
    clf.add_task(task_id, X_train, y_train)

clf.fit([0, 1])
for task_id in task_data:
    X_test = task_data[task_id][1]
    y_test = task_data[task_id][3]
    acc = clf.score(X_test, y_test, task_id=task_id)
    print(f"Task {task_id} (Spiral {3 if task_id==0 else 5}) Baseline Accuracy: {acc:.3f}")


Task 0 (Spiral 3) Baseline Accuracy: 0.740
Task 1 (Spiral 5) Baseline Accuracy: 0.740


###Forward training, backward training, reverse training

In [29]:
results = clf.evaluate_transfer_general(
    forward_train_ids=[0],
    forward_test_id=1,
    backward_train_ids=[0, 1],
    backward_test_ids=[0, 1],
    do_reverse=True
)

fwd = results["forward_transfer"]
print(f"\nForward Transfer:\n  Train on: {fwd['train_on']}\n  Test on: {fwd['test_on']}\n  Accuracy: {fwd['accuracy']:.3f}")

print("\nBackward Transfer:")
for task, stats in results["backward_transfer"].items():
    print(f"  {task}:")
    print(f"    Train on: {stats['train_on']}")
    print(f"    Test on : {stats['test_on']}")
    print(f"    Accuracy: {stats['accuracy']:.3f}")

if "reverse_transfer" in results:
    print("\nReverse Transfer:")
    for task, stats in results["reverse_transfer"].items():
        print(f"  {task}:")
        print(f"    Train on: {stats['train_on']}")
        print(f"    Test on : {stats['test_on']}")
        print(f"    Accuracy: {stats['accuracy']:.3f}")


Forward Transfer:
  Train on: [0]
  Test on: 1
  Accuracy: 0.160

Backward Transfer:
  task0:
    Train on: [0, 1]
    Test on : 0
    Accuracy: 0.960
  task1:
    Train on: [0, 1]
    Test on : 1
    Accuracy: 0.982

Reverse Transfer:
  task0:
    Train on: [1]
    Test on : 0
    Accuracy: 0.218
